In [ ]:
import logging
import traceback
import re
import yaml
from google.cloud import bigquery
from google.cloud import storage
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound

# Configuración del proyecto y dataset de BigQuery
PROJECT_ID = 'ultimate-realm-388922'
BQ_DATASET = 'taxis_nyc'  # Nombre del dataset

# Carga la configuración de esquemas desde el archivo YAML
with open("tripdata_schema.yaml") as schema_file:
    config = yaml.load(schema_file, Loader=yaml.FullLoader)

# Inicializa clientes para Cloud Storage y BigQuery
storage_client = storage.Client()
bigquery_client = bigquery.Client()

# Configuración inicial para los trabajos de carga en BigQuery
job_config = bigquery.LoadJobConfig()

# Las funciones auxiliares van aquí (check_and_create_table, load_parquet_into_bigquery, create_bigquery_schema)
def check_and_create_table(table_name, table_schema):
    """ Verifica si una tabla existe en BigQuery y la crea si no existe. """
    table_id = f"{PROJECT_ID}.{BQ_DATASET}.{table_name}"

    try:
        bigquery_client.get_table(table_id)
    except NotFound:
        logging.warning(f'Creating table: {table_name}')
        schema = create_bigquery_schema(table_schema)
        table = bigquery.Table(table_id, schema=schema)
        bigquery_client.create_table(table)
        print(f"Created table {table_id}")

def load_parquet_into_bigquery(bucket_name, file_name, table_schema, table_name):
    """ Carga un archivo Parquet desde Cloud Storage a una tabla de BigQuery. """
    uri = f'gs://{bucket_name}/{file_name}'
    table_id = f"{PROJECT_ID}.{BQ_DATASET}.{table_name}"

    schema = create_bigquery_schema(table_schema)
    job_config.schema = schema
    job_config.source_format = bigquery.SourceFormat.PARQUET
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

    load_job = bigquery_client.load_table_from_uri(uri, table_id, job_config=job_config)
    load_job.result()
    print(f"La tarea de carga para {uri} hacia {table_id} ha finalizado.")

def create_bigquery_schema(table_schema_yaml):
    """ Convierte el esquema YAML a un esquema de BigQuery. """
    schema = []
    for column in table_schema_yaml:
        schema_field = bigquery.SchemaField(column['name'], column['type'], column.get('mode', 'NULLABLE'))
        schema.append(schema_field)
        if column['type'] == 'RECORD':
            schema_field._fields = create_bigquery_schema(column['fields'])
    return schema

def gcf_trigger(data, context):
    """ Función para procesar un archivo subido a Cloud Storage y cargarlo en BigQuery. """
    bucket_name = data['bucket']
    file_name = data['name']
    time_created = data['timeCreated']

    print(f"Bucket: {bucket_name}, File: {file_name}, Time Created: {time_created}")

    # Comprobar si el archivo está en una de las carpetas específicas
    if (file_name.startswith('pre-data/taxis-amarillos/') or 
        file_name.startswith('pre-data/taxis-verdes/') or 
        file_name.startswith('pre-data/taxis-alquiler/')):
        try:
            for table_config in config:
                table_name = table_config['name']  # Asegúrate de que config es una lista de diccionarios
                if re.search(table_name.replace('_', '-'), file_name) or re.search(table_name, file_name):
                    table_schema = table_config['schema']
                    check_and_create_table(table_name, table_schema)
                    load_parquet_into_bigquery(bucket_name, file_name, table_schema, table_name)
        except Exception as e:
            print(f'Error al procesar el archivo: {traceback.format_exc()}')
    else:
        print(f"Ignorando archivo fuera de las carpetas especificadas: {file_name}")
